In [3]:
import os
import pandas as pd
import pymysql
import numpy as np
from haversine import haversine
from decimal import Decimal
from tqdm.auto import tqdm

tqdm.pandas()

def dev_db_conn():
    """개발 DB 접속 함수"""
    conn = pymysql.connect(
        host = 'db-7ma06.pub-cdb.ntruss.com',
        user = 'redtable',
        password = 'fpemxpdlqmf5491!@#',
        autocommit = True,
        cursorclass= pymysql.cursors.DictCursor)
    return conn


def rt_db_conn():
    """빅데이터센터 DB 접속 함수"""
    conn = pymysql.connect(
        host = '220.95.212.6',
        user = 'red_dev_jiang',
        password = 'redtable211b',
        db = 'redtable_bigdata_center',
        autocommit = True,
        cursorclass= pymysql.cursors.DictCursor)
    conn.query("set character_set_connection=utf8;")
    conn.query("set character_set_server=utf8;")
    conn.query("set character_set_client=utf8;")
    conn.query("set character_set_results=utf8;")
    conn.query("set character_set_database=utf8;")
    return conn


file_path = os.path.join(os.environ["HOMEPATH"], "Downloads/")

## 지방공항 주변 식당 데이터 데이터
## RT_LOCAL_ARPRT_RSTRNT_INFO_202407
## https://www.bigdata-culture.kr/bigdata/user/data_market/detail.do?id=f4843e60-3b97-11eb-af9a-4b03f0a582d6

In [4]:
def text_extract(input_text):
    """상세주소 제외한 함수"""
    result_text = " ".join(input_text.split()[:-1])
    if result_text.endswith("리"):
        return " ".join(result_text.split()[:-1])
    return result_text


# 공항데이터 추가 (2024-07-09)
data = {
    '0':["1","거점공항","김포국제공항","서울특별시 강서구 방화동 886","서울특별시 강서구 하늘길 38","11540232",37.5668093694949,126.802955054368],
    '1':["2","거점공항","김해국제공항","부산광역시 강서구 대저2동 2350","부산광역시 강서구 공항진입로 108","13036655",35.1728739832702,128.945800032246],
    '2':["3","거점공항","제주국제공항","제주특별자치도 제주시 용담이동 2002","제주특별자치도 제주시 공항로 2","11576965",33.5077816117418,126.49313255015],
    '3':["4","거점공항","대구국제공항","대구광역시 동구 지저동 400-1","대구광역시 동구 공항로 221","11576983",35.8998531330912,128.639589769784],
    '4':["5","거점공항","청주국제공항","충청북도 청주시 청원구 내수읍 입상리 산5-4","충청북도 청주시 청원구 내수읍 오창대로 980","11576967",36.7254221183578,127.495739144007],
    '5':["6","거점공항","무안국제공항","전라남도 무안군 망운면 피서리 1065","전라남도 무안군 망운면 공항로 970-260","12301846",34.9961732910783,126.387631382857],
    '6':["7","일반공항","양양국제공항","강원특별자치도 양양군 손양면 동호리 545","강원특별자치도 양양군 손양면 공항로 201","11576985",38.0588534211383,128.662845957111],
    '7':["8","일반공항","울산공항","울산광역시 북구 화봉동 522","울산광역시 북구 산업로 1103","11576970",35.593546098694,129.356473479956],
    '8':["9","일반공항","여수공항","전라남도 여수시 율촌면 신풍리 979","전라남도 여수시 율촌면 여순로 386","11576974",34.8418229412521,127.612794176105],
    '9':["10","일반공항","사천공항","경상남도 사천시 사천읍 구암리 1713","경상남도 사천시 사천읍 사천대로 1971","11576975",35.0922446803508,128.086689281509],
    '10':["11","일반공항","포항경주공항","경상북도 포항시 남구 동해면 도구리 402-1","경상북도 포항시 남구 동해면 일월로 18","11576976",35.9841133319753,129.434039019476],
    '11':["12","일반공항","군산공항","전북특별자치도 군산시 옥서면 선연리 385","전북특별자치도 군산시 옥서면 산동길 2","11576978",35.9264364527116,126.615614887301],
    '12':["13","일반공항","원주공항","강원특별자치도 횡성군 횡성읍 곡교리 112","강원특별자치도 횡성군 횡성읍 횡성로 42","11576981",37.4592489036448,127.976445307279],
    '13':["14","일반공항","광주공항","광주광역시 광산구 신촌동 698-9","광주광역시 광산구 상무대로 420-25","11576966",35.13938861719,126.810912274867]
}
column_list = ['SN','항목','이름','지번주소','도로명주소','naver_id', '위도', '경도']
df = pd.DataFrame.from_dict(data=data, orient='index', columns=column_list)
df["검색주소"] = df["지번주소"].apply(lambda x:text_extract(x))

In [10]:
for i in range(len(df)):
    with dev_db_conn() as conn:
        with conn.cursor() as curs:
            sql = f"""
            SELECT T3.SRC_ID AS 'RSTRNT_ID', %s AS 'ARPRT_NM', T1.RSTR_NM AS 'RSTRNT_NM',
                IF(T1.RSTR_LNNO_ADRES = '', NULL, T1.RSTR_LNNO_ADRES) AS 'RSTRNT_LNM_ADDR',
                IF(T1.RSTR_RDNMADR = '', NULL, T1.RSTR_RDNMADR) AS 'RSTRNT_ROAD_NM_ADDR',
                '' AS 'RSTRNT_TO_DSTNC', IF(T1.RSTR_TELNO = '', NULL, REPLACE(T1.RSTR_REPRSNT_TELNO, '-','')) AS 'RSTRNT_TEL_NO',
                T1.RSTR_LA AS 'RSTRNT_LA', T1.RSTR_LO AS 'RSTRNT_LO', 'KOR' AS 'COUNTRY_NM',
                REPLACE(REPLACE(REPLACE(T1.AREA_NM_1, '강원도', '강원특별자치도'), '전라북도', '전북특별자치도'), '제주도', '제주특별자치도') AS 'CTY_NM'
            FROM redtable01.RSTR T1
            INNER JOIN redtable01.RSTR_INDEX T2 ON T1.ID = T2.RSTR_ID
            INNER JOIN redtable01.RSTR_CONN T3 ON T2.RSTR_ID = T3.RSTR_ID
            WHERE T2.IS_USE=1 AND T2.IS_OPEN=1 AND T3.SRC='live_db' AND T1.AREA_NM_1 IS NOT NULL AND T1.AREA_NM_1<>''
            AND T1.RSTR_NM IS NOT NULL AND T1.RSTR_NM<>'' AND T1.RSTR_NM NOT LIKE '%%？%%'
            AND T1.RSTR_LNNO_ADRES LIKE %s
            GROUP BY T3.SRC_ID;
            """
            curs.execute(sql, (df["이름"][i], f'%{df["검색주소"][i]}%'))
            df_result = pd.DataFrame(curs.fetchall())
            
    if i == 0:
        df_final = df_result
    
    else:
        df_final = pd.concat([df_final, df_result])
        
# 결측치 및 불필요한 데이터 정리
df_final = df_final.fillna(np.nan)
df_final.replace({np.nan:None, '\\n':None}, inplace=True)

In [11]:
# 공항 위경도 붙이기
set_idx = df.set_index("이름")["위도"].to_dict()
df_final["공항위도"] = df_final["ARPRT_NM"].map(set_idx)

set_idx = df.set_index("이름")["경도"].to_dict()
df_final["공항경도"] = df_final["ARPRT_NM"].map(set_idx)

df_final.reset_index(drop=True, inplace=True)

def haversine_calculate(store_la,store_lo,airport_la,airport_lo):
    """위경도로 거리 검색하는 함수"""
    try:
        store = (float(store_la), float(store_lo))
        ariport = (float(airport_la), float(airport_lo))
        return haversine(store, ariport, unit = 'm')
    except:
        return 0

# 함수를 통해 위경도로 거리 검색하기 (단위 m)
df_final["RSTRNT_TO_DSTNC"] = df_final.progress_apply(lambda row:
                                                      (haversine_calculate(row["RSTRNT_LA"],row["RSTRNT_LO"]
                                                                           ,row["공항위도"],row["공항경도"]) ),axis=1).tolist()

del df_final["공항위도"]
del df_final["공항경도"]

# 주소가 이상한 결과 필터링
df_final = df_final[df_final["RSTRNT_TO_DSTNC"] != 0]

  0%|          | 0/3692 [00:00<?, ?it/s]

In [12]:
df_final

,RSTRNT_ID,ARPRT_NM,RSTRNT_NM,RSTRNT_LNM_ADDR,RSTRNT_ROAD_NM_ADDR,RSTRNT_TO_DSTNC,RSTRNT_TEL_NO,RSTRNT_LA,RSTRNT_LO,COUNTRY_NM,CTY_NM
0,762455,김포국제공항,물망초,서울특별시 강서구 방화동 620-33,서울특별시 강서구 방화동로3길 6,686.846247,026638047,37.5626582,126.8087256,KOR,서울특별시
1,762462,김포국제공항,충북집,서울특별시 강서구 방화동 620-52,서울특별시 강서구 개화동로31길 16,661.722821,026662418,37.5625010,126.8081340,KOR,서울특별시
2,51337,김포국제공항,육대장,서울특별시 강서구 방화동 621-12,서울특별시 강서구 방화동로 1,684.155599,0263268600,37.5621636,126.8080442,KOR,서울특별시
3,762491,김포국제공항,왕궁,서울특별시 강서구 방화동 620-33,서울특별시 강서구 방화동로3길 6,686.846247,None,37.5626582,126.8087256,KOR,서울특별시
4,51341,김포국제공항,수정커피숍,서울특별시 강서구 방화동 564-44,서울특별시 강서구 방화동로 119,929.835631,0226651410,37.5709569,126.8121160,KOR,서울특별시
...,...,...,...,...,...,...,...,...,...,...,...
3687,989251,광주공항,드롭탑광주공항점,광주광역시 광산구 신촌동 698-9,광주광역시 광산구 상무대로 420-25,0.002979,None,35.1393886,126.8109123,KOR,광주광역시
3688,1153746,광주공항,라이크펫(LikePet),광주광역시 광산구 신촌동 1080-2,광주광역시 광산구 신흥동안길 30,1216.081775,None,35.1458679,126.8001378,KOR,광주광역시
3689,1165648,광주공항,광주하림비엔에프,광주광역시 광산구 신촌동 681-20,광주광역시 광산구 상무대로419번길 8-26,687.092086,0624569999,35.1445940,126.8149840,KOR,광주광역시
3690,1168690,광주공항,한결식품,광주광역시 광산구 신촌동 954-1,광주광역시 광산구 사암로 30-6,867.977568,0624490239,35.1467730,126.8078178,KOR,광주광역시


In [13]:
with rt_db_conn() as conn:
    sql = """
    SELECT TABLE_NAME, COLUMN_NAME, column_comment
    FROM information_schema.`COLUMNS`
    WHERE TABLE_NAME = 'RT_LOCAL_ARPRT_RSTRNT_INFO_202407'
    """
    columns = pd.read_sql(sql, conn)
    
# 컬럼명 가져오기
db_cols = ','.join(columns['COLUMN_NAME'])

In [14]:
with rt_db_conn() as conn:
    with conn.cursor() as curs:
        sql = f"""
        INSERT IGNORE INTO redtable_bigdata_center.RT_LOCAL_ARPRT_RSTRNT_INFO_202407({db_cols})
        VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
        """
        val = df_final.apply(lambda row:(row["RSTRNT_ID"], row["ARPRT_NM"], row["RSTRNT_NM"], row["RSTRNT_LNM_ADDR"],
                                  row["RSTRNT_ROAD_NM_ADDR"], row["RSTRNT_TO_DSTNC"], row["RSTRNT_TEL_NO"], row["RSTRNT_LA"],
                                  row["RSTRNT_LO"], row["COUNTRY_NM"], row["CTY_NM"]), axis=1).tolist()
        curs.executemany(sql, val)